In [2]:
import json
import os
import logging
import argparse
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

/dcs/pg23/u5579267/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class ModelPredictor:
    def __init__(self, model_dir, label_mapping, use_cpu=False):
        self.model_dir = model_dir
        self.label_mapping = label_mapping
        self.use_cpu = use_cpu

        logger.info("Loading the model and tokenizer.")
        device = torch.device("cuda" if torch.cuda.is_available() and not self.use_cpu else "cpu")

        self.tokenizer = AutoTokenizer.from_pretrained(model_dir)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)
        self.device = device

        # Create reverse label mapping
        self.reverse_label_mapping = {v: k for k, v in label_mapping.items()}

    def predict(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)
        
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        predicted_label = self.reverse_label_mapping[preds[0]]

        return predicted_label

In [4]:
model_dir = os.path.join(os.getcwd(), 'flanT5-baseModel/final_model')
use_cpu = True

label_mapping = {'I-A': 0, 'I-CT': 1, 'I-DC': 2, 'I-DI': 3, 'I-GC': 4,
                 'I-NC': 5, 'I-RD': 6, 'I-RSS': 7, 'I-SD': 8, 'I-SR': 9, 'I-SS': 10, 'O': 11}

In [5]:
input_text = "Frisch's Restaurants , Inc . Clarifies Payment of Special Dividend CINCINNATI , Aug . 22 , 2012 / / As previously announced , Frisch's Restaurants , Inc . ( NYSE MKT: FRS ) declared a special one-time cash dividend of $9.50 per share payable on September 14 , 2012 , to shareholders of record on August 31 , 2012 . This dividend is taxable as an ordinary dividend and not as a return on capital . For material special dividends , the rules of the New York Stock Exchange require the ex-dividend date to be one business day after the payment date . Accordingly , the ex-dividend date has been set for Monday , September 17 , 2012 . Shares of the Company's stock will trade with \"due-bills\" beginning August 29 , 2012 ( two business days prior to the record date ) through the September 14 , 2012 payment date . Shareholders who sell their shares before the payment date will not be entitled to receive the special cash dividend from the Company . Due-bills obligate sellers ( who were shareholders of record on August 31 ) to deliver the dividend to the buyer . The due-bill obligations are settled customarily between the brokers representing the buyers and sellers of the stock . Frisch's has no obligations for either the amount of the due-bill or the processing of the due-bill . About Frisch's Restaurants , Inc . Frisch's is a regional company that operates full service family-style restaurants under the name \"Frisch's Big Boy.\" All Big Boy restaurants are currently located in various regions of Ohio , Kentucky and Indiana . The Company owns the trademark \"Frisch's\" and has exclusive , irrevocable ownership of the rights to the \"Big Boy\" trademark , trade name and service marks in the states of Kentucky and Indiana , and in most of Ohio and Tennessee . All of the Frisch's Big Boy restaurants also offer \"drive-thru\" service . The Company also licenses Big Boy restaurants to other operators , currently in certain parts of Ohio , Kentucky and Indiana . Statements contained in this press release which are not historical facts are forward looking statements as that item is defined in the Private Securities Litigation Act of 1995 . Such forward looking statements are subject to risks and uncertainties which could cause actual results to differ materially from estimated results . Such risks and uncertainties are detailed in the company's filings with the Securities and Exchange Commission . SOURCE Frisch's Restaurants , Inc."

In [6]:
predictor = ModelPredictor(model_dir=model_dir, label_mapping=label_mapping, use_cpu=use_cpu)
prediction = predictor.predict(input_text)

print(f"Extracted Event: {prediction}")

INFO:__main__:Loading the model and tokenizer.


Extracted Event: I-SD


In [7]:
input_text = "XYZ Pharmaceuticals, Inc. Announces Acquisition of ABC Biotech Corp. NEW YORK, June 15, 2023 // XYZ Pharmaceuticals, Inc. (NASDAQ: XYZP) has entered into a definitive agreement to acquire ABC Biotech Corp., a leading biotechnology company focused on innovative cancer therapies, for $1.2 billion in an all-cash transaction. The acquisition is expected to close in the fourth quarter of 2023, subject to customary closing conditions, including regulatory approvals and approval by ABC Biotech Corp.'s shareholders. The acquisition of ABC Biotech Corp. is expected to enhance XYZ Pharmaceuticals' oncology portfolio and accelerate the development of ABC's promising pipeline of cancer treatments. This transaction represents a strategic opportunity for XYZ Pharmaceuticals to expand its market presence in the oncology sector and drive long-term growth. Statements contained in this press release which are not historical facts are forward-looking statements as defined in the Private Securities Litigation Act of 1995. These statements are subject to risks and uncertainties which could cause actual results to differ materially from those anticipated. For more details, please refer to the company's filings with the Securities and Exchange Commission. SOURCE: XYZ Pharmaceuticals, Inc."

In [8]:
predictor = ModelPredictor(model_dir=model_dir, label_mapping=label_mapping, use_cpu=use_cpu)
prediction = predictor.predict(input_text)

print(f"Extracted Event: {prediction}")

INFO:__main__:Loading the model and tokenizer.


Extracted Event: I-A


In [9]:
input_text = "Alpha Tech Corp. Announces 2-for-1 Stock Split SAN FRANCISCO, April 10, 2024 // Alpha Tech Corp. (NYSE: ATC) announced today that its Board of Directors has approved a 2-for-1 stock split of the company's common stock. The stock split will be effected in the form of a stock dividend, where shareholders will receive one additional share for each share held as of the close of business on May 1, 2024. The additional shares will be distributed on May 15, 2024, and trading will begin on a split-adjusted basis on May 16, 2024. This stock split is intended to make the stock more accessible to a broader base of investors by reducing the share price and increasing liquidity. Shareholders do not need to take any action to receive the additional shares, as they will be automatically credited to their accounts. Statements contained in this press release which are not historical facts are forward-looking statements as defined in the Private Securities Litigation Act of 1995. Such statements involve risks and uncertainties that may cause actual results to differ materially. For further details, please refer to the company's filings with the Securities and Exchange Commission. SOURCE: Alpha Tech Corp."

In [10]:
predictor = ModelPredictor(model_dir=model_dir, label_mapping=label_mapping, use_cpu=use_cpu)
prediction = predictor.predict(input_text)

print(f"Extracted Event: {prediction}")

INFO:__main__:Loading the model and tokenizer.


Extracted Event: I-SS


In [11]:
input_text = """
TerrAscend authorizes $10M share buyback program

Aug. 20, 2024 9:10 AM ETTerrAscend Corp. (TSNDF) Stock, TSND:CA StockBy: Sinchita Mitra, SA News Editor

Play
(<1min)
TerrAscend (OTCQX:TSNDF) said its board has authorized the company to commence a normal course issuer bid to repurchase up to $10 million of its common shares from time to time over a 12-month period.
The firm is authorized to buyback up to 10M of its shares.
"""

In [12]:
predictor = ModelPredictor(model_dir=model_dir, label_mapping=label_mapping, use_cpu=use_cpu)
prediction = predictor.predict(input_text)

print(f"Extracted Event: {prediction}")

INFO:__main__:Loading the model and tokenizer.


Extracted Event: I-SR


In [13]:
input_text = """
MGE Energy raises dividend by 5.3% to $0.45
Aug. 20, 2024 8:22 AM ETMGE Energy, Inc. (MGEE) StockBy: Deepa Sarvaiya, SA News Editor

Play
(<1min)
MGE Energy (NASDAQ:MGEE) declares $0.45/share quarterly dividend, 5.3% increase from prior dividend of $0.43.
Forward yield 2.05%
Payable Sept. 15; for shareholders of record Sept. 1; ex-div Aug. 30.
"""

In [14]:
predictor = ModelPredictor(model_dir=model_dir, label_mapping=label_mapping, use_cpu=use_cpu)
prediction = predictor.predict(input_text)

print(f"Extracted Event: {prediction}")

INFO:__main__:Loading the model and tokenizer.


Extracted Event: I-RD


In [ ]:
input_text = """
The FTSE 100 high-street retailer reported that full-price sales in the second quarter were up 3.2% year-on-year, exceeding expectations by £42m.

That was particularly notable given the exceptional summer weather last year, which had led to a forecasted decline of 0.3% for the second quarter.

For the half-year, full-price sales rose by 4.4% compared to the same period last year, surpassing the company’s guidance of a 2.5% increase.

Total group sales, which include markdowns, subsidiaries, and investments, grew 8% in the half.

The growth was bolstered by the acquisition of FatFace and an increased shareholding in Reiss, both of which were finalised in the third quarter of last year.

Next put its profit upgrade down to additional sales contributing £11m and cost savings of £9m, primarily in logistics.

Full-price sales in the UK, combining online and retail, were slightly above expectations, rising by 0.4%.

Overseas online sales significantly outperformed, with a 21.9% increase.

Next also reported a return to more typical levels of surplus stock for its end-of-season sale, following a particularly low surplus last year.

The surplus stock was up 21% compared to last year but remained 5% below the levels from two years ago, with clearance rates aligning with internal forecasts.

Looking ahead, Next maintained its guidance for second-half full-price sales to rise by 2.5% year-on-year.

That forecast, although appearing cautious compared to the first half’s 4.4% growth, was consistent with growth figures from two years ago.

The company said it expected total group sales for the full year to increase by 6.0%, with a 3.4% rise in Next trading full-price sales, augmented by the contributions from recent acquisitions.

Next said it would announce its interim results for the first half on 19 September.

At 0912 BST, shares in Next were up 8.22% at 9,824p.
"""